In [1]:
import os

In [2]:
os.getcwd()

'c:\\TheBritishCollege\\DataScience\\Wafer_sensor_project\\research'

In [3]:
os.chdir("../")

In [14]:
from dataclasses import dataclass
from pathlib import Path

In [26]:
@dataclass(frozen=True)
class DataIngestionConfig: 
    root_dir: Path
    data_source: Path
    data_source: Path
    train_data_path: Path
    test_data_path: Path
    data_to_store: Path
    

In [9]:
from wafer_project.utils.common import read_yaml, create_directories
from wafer_project.constant import *

In [27]:

class ConfigurationManager: 
    def __init__(self, config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH
                 ): 
        self.config=read_yaml(config_file_path)
        self.params=read_yaml(params_file_path)
        self.schema=read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self)-> DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
         
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            data_source=config.data_source,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            data_to_store=config.data_to_store)
        
        return data_ingestion_config
    

In [36]:
import pandas as pd
import numpy as np
from wafer_project import logger
from sklearn.model_selection import train_test_split

In [38]:
class DataIngestion: 
    def __init__(self, config=DataIngestionConfig): 
        self.config=config
        
    def initiate_data_ingestion(self): 
        try: 
            path=Path("Data\wafer_dataset.csv")
            data=pd.read_csv(path)
            logger.info(f"Dataset : {data.head(2)}")
            data=data.drop("Unnamed: 0", axis=1)
            logger.info(f"Deleting the column-- Unnamed: 0")
            
            data.to_csv(self.config.data_to_store, index=False, header=True)
            
            (train_data, test_data)=train_test_split(data, test_size=0.2, random_state=42)
            
            train_data.to_csv(self.config.train_data_path, index=False, header=True)
            logger.info(f"Train data {train_data.head(2)}")
            
            test_data.to_csv(self.config.test_data_path, index=False, header=True)
            logger.info(f"Test data {test_data.head(2)}")
            
        except Exception as e: 
            raise e
        

In [42]:
config_manager=ConfigurationManager()
ingestion_config=config_manager.get_data_ingestion_config()
dataIngestion=DataIngestion(ingestion_config)
dataIngestion.initiate_data_ingestion()


[2023-12-20 17:25:08,273 : INFO : common : Yaml file read sucessfully: config\config.yaml]
[2023-12-20 17:25:08,279 : INFO : common : Yaml file read sucessfully: params.yaml]
[2023-12-20 17:25:08,283 : INFO : common : Yaml file read sucessfully: schema.yaml]
[2023-12-20 17:25:08,286 : INFO : common :  Directories is created : artifacts]
[2023-12-20 17:25:08,292 : INFO : common :  Directories is created : artifacts/data_ingestion]


[2023-12-20 17:25:08,488 : INFO : 3315771610 : Dataset :   Unnamed: 0  Sensor-1  Sensor-2   Sensor-3   Sensor-4  Sensor-5  Sensor-6  \
0  Wafer-801   2968.33   2476.58  2216.7333  1748.0885    1.1127     100.0   
1  Wafer-802   2961.04   2506.43  2170.0666  1364.5157    1.5447     100.0   

   Sensor-7  Sensor-8  Sensor-9  ...  Sensor-582  Sensor-583  Sensor-584  \
0   97.5822    0.1242    1.5300  ...         NaN      0.5004      0.0120   
1   96.7700    0.1230    1.3953  ...         NaN      0.4994      0.0115   

   Sensor-585  Sensor-586  Sensor-587  Sensor-588  Sensor-589  Sensor-590  \
0      0.0033      2.4069      0.0545      0.0184      0.0055     33.7876   
1      0.0031      2.3020      0.0545      0.0184      0.0055     33.7876   

   Good/Bad  
0        -1  
1         1  

[2 rows x 592 columns]]
[2023-12-20 17:25:08,493 : INFO : 3315771610 : Deleting the column-- Unnamed: 0]
[2023-12-20 17:25:08,718 : INFO : 3315771610 : Train data     Sensor-1  Sensor-2   Sensor-3   Senso